In [1]:
import getpass
import os
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

c:\Users\ryanz\Desktop\LLM-Based-NPC\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in English.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

from langchain_core.messages import SystemMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

from typing import Literal

class State(MessagesState):
    summary: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    # If a summary exists, we add this in as a system message
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
        
    #Chain prompt template to model
    chain = prompt | model
    response = chain.invoke(
        {"messages": messages}
    )
    
    response = model.invoke(messages)
    return {"messages": [response]}

def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """Return the next node to execute."""
    messages = state["messages"]
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    # Otherwise we can just end
    return END

def summarize_conversation(state: State):
    # First, we summarize the conversation
    summary = state.get("summary", "")
    if summary:
        # If a summary already exists, we use a different system prompt
        # to summarize it than if one didn't
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # We now need to delete messages that we no longer want to show up
    # I will delete all but the last two messages, but you can change this
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
workflow.add_node(summarize_conversation)

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `conversation`.
    # This means these are the edges taken after the `conversation` node is called.
    "model",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

workflow.add_edge("summarize_conversation", END)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [4]:
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])
            
            
config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="hi! I'm bob")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="i like the celtics!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello Bob, I'm Command. How can I help you today?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob. Is there anything else I can help you with?
================================ Human Message =================================

i like the celtics!
================================== Ai Message ==================================

The Celtics are a professional basketball team based in Boston, Massachusetts. They compete in the National Basketball Association (NBA) as part of the league's Eastern Conference Atlantic Division. The Celtics have won a record 17 NBA championships, the most of any NBA franchise.

Would you like to know more about the Celtics?


In [4]:
input_message = HumanMessage(content="i like how much they win")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like how much they win
================================== Ai Message ==================================

The Celtics have indeed been a dominant force in the NBA, especially during the 1950s and 1960s, when they won 11 championships in 13 years, a feat that has not been matched since. This era was led by legendary players like Bob Cousy, Bill Russell, and Tom Heinsohn, who helped establish the Celtics' winning culture.

In recent years, the Celtics have continued their tradition of success, reaching the NBA Finals in 2008, 2010, and 2022, winning the championship in 2008. The team's success can be attributed to a combination of skilled players, strategic coaching, and a strong front office.

If you'd like, I can provide more details about the Celtics' recent seasons and notable players.
================================ Remove Message ================================


================================ Re

In [9]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='my name is bob.', additional_kwargs={}, response_metadata={}, id='5bf33737-5508-4c05-b64e-a71d6527cc7e'),
  AIMessage(content='I remember, Bob. Is there anything else I can help you with today?', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'e355e40a-5141-404e-99d1-c70f4615fb4c', 'token_count': {'input_tokens': 390.0, 'output_tokens': 16.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'e355e40a-5141-404e-99d1-c70f4615fb4c', 'token_count': {'input_tokens': 390.0, 'output_tokens': 16.0}}, id='run-e0895070-2eb8-4e77-b1b1-4e70d55ffb5e-0', usage_metadata={'input_tokens': 390, 'output_tokens': 16, 'total_tokens': 406}),
  HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}, id='74b9f901-6642-4646-a816-19753d11f595'),

In [10]:

input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob, as you introduced yourself earlier in our conversation. Is there anything else you'd like to know or discuss? I'm here to assist you with any further inquiries.


In [5]:
input_message = HumanMessage(content="what's my name?")
output = app.invoke({"messages": [input_message]}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I apologize; I don't have access to personal information, including your name. In our previous conversation, you were referred to as Bob, but if that's not your name, please let me know how you'd like to be addressed, and I'll be happy to use that name in our discussion about the Boston Celtics or any other topic you'd like to explore.


In [6]:
input_message = HumanMessage(content="my name is bob.")
output = app.invoke({"messages": [input_message]}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I remember, Bob. Is there anything else I can help you with today?


In [7]:
input_message = HumanMessage(content="Whats your name?")
output = app.invoke({"messages": [input_message]}, config)
output["messages"][-1].content

'My name is Command. I am an AI assistant developed by Cohere. I am designed to assist you with various tasks and provide thorough responses to your queries. If you have any specific questions or need assistance with a particular topic, feel free to ask!'